# STATO OWL with OWLAPI 

Here are examples of using [OWLAPI](http://owlcs.github.io/owlapi/) to process [STATO, the statistical methods ontology](https://www.ebi.ac.uk/ols/ontologies/stato). 

In [4]:
%%loadFromPOM
<!-- Use %%loadFromPOM instead of %maven. For some reason, the latter does not load guava correctly. -->
<dependency>
    <groupId>net.sourceforge.owlapi</groupId>
    <artifactId>owlapi-distribution</artifactId>
    <version>5.1.11</version>
</dependency>

In [9]:
import org.semanticweb.owlapi.apibinding.OWLManager;
import org.semanticweb.owlapi.model.*;
import org.semanticweb.owlapi.vocab.DublinCoreVocabulary;
import org.semanticweb.owlapi.vocab.OWLRDFVocabulary;

In [6]:
import java.net.URI;
URI uri = new URI("http://purl.obolibrary.org/obo/stato.owl");
IRI iri = IRI.create(uri);

In [7]:
// load the ontology
OWLOntologyManager manager = OWLManager.createOWLOntologyManager();
OWLOntology onto = manager.loadOntology(iri);

In [12]:
// Retreive annotations
onto.annotations().forEach((ann) -> {
    IRI propIRI = ann.getProperty().getIRI();
    OWLLiteral value = (OWLLiteral) ann.getValue();
    if (propIRI.equals(DublinCoreVocabulary.TITLE.getIRI())) {
        System.out.println("Title: " + value.getLiteral());
    } else if (propIRI.equals(OWLRDFVocabulary.OWL_VERSION_INFO.getIRI())) {
        System.out.println("Version: " + value.getLiteral());
    }
});

Title: STATO: the statistical methods ontology
Version: RC1.4
